In [1]:
from scipy.special import j1
from scipy.optimize import brentq
from sage.all import *
import struct

DR = RealField(52)

DD = RealField(157)

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")


In [2]:
# from scipy.special import j1, jvp
# from scipy.optimize import brentq
from mpmath import mp, mpf, findroot, j1, besselj
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")
    
# Use Sage's RDF for high-precision floats
R120 = RealField(120)

# List of zeros
zeros = []

# Step size to detect sign changes
# step = R120(0.001)
# x = R120(0.0)

mp.prec = 200

step = mpf("0.01")
epsilon = mpf("1e-35")
x = mpf("0.0")

def j1_prime(x):
    return diff(lambda t: besselj(1, t), x)

previous_zero = R120(0)
# epsilon = 1e-12
j1_zeros = []

# while x < 77:
#     if j1(float(x)) * j1(float(x + step)) < 0:  # Sign change => zero in interval
#         previous_zero = brentq(j1, x, x + step)
#         j1_zeros.append(R120(previous_zero))
#     if abs(x - round(x)) < epsilon:
#         zeros.append(R120(previous_zero))
#     x += step

while x < mpf("76.0"):
    f1 = besselj(1, x)
    f2 = besselj(1, x + step)
    if f1 * f2 < 0:
        zero = findroot(lambda t: j1(t), (x, x + step), solver='bisect', tol=mp.mpf("1e-41"))
        previous_zero = zero
        j1_zeros.append(zero)
    if previous_zero is not None and abs(x - mpf(f'{round(x)}')) < epsilon:
        zeros.append(previous_zero)
    x += step

j1_extrema = []

# x = R120(0.0)
# while x < 77:
#     dfx = jvp(1, float(x), n=1)  # Derivative of J1
#     dfx_next = jvp(1, float(x + step), n=1)
#     if dfx * dfx_next < 0:
#         extremum = brentq(lambda x: jvp(1, x, n=1), float(x), float(x + step))
#         j1_extrema.append(R120(extremum))
#     x += step

x = mpf("0.0")
while x < mpf("76.0"):
    d1 = mp.diff(lambda t: j1(t), x)
    d2 = mp.diff(lambda t: j1(t), x + step)
    if d1 * d2 < 0:
        extremum = findroot(lambda t: mp.diff(lambda u: j1(u), t), (x, x + step), solver='bisect', tol=mp.mpf("1e-41"))
        j1_extrema.append(extremum)
    x += step

# Print results
for i, z in enumerate(j1_zeros):
    print(f"Zero {i+1}: x ≈ {z}")

print("Extrema (peaks/valleys) of J1(x):")
for e in j1_extrema:
    print(f"nExtrema: {e}")

j1_zeros.extend(j1_extrema)

j1_zeros = sorted(j1_zeros)

# Print results
for i, z in enumerate(j1_zeros):
    print(f"Peak or zero {i+1}: x ≈ {z}")

print("")

print("pub(crate) static J1_ZEROS: [(u64, u64); 48] = [")
print(f"(0x0, 0x0),")
for z in j1_zeros:
    k = split_double_double(z)
    # print_dyadic(z)
    hi = double_to_hex(k[1])
    lo = double_to_hex(k[0])
    print(f"({lo}, {hi}),")
    
print("];")

from mpmath import *
from sage.all import *

from mpmath import mp, j1, taylor, expm1

prev_zero = 0

CDR = RealField(120)

var('x')

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffs(poly, x0):
    print("[")
    for i in range(0, 24):
        coeff = poly[i]
        print_double_double("", coeff)
        # print(f"{double_to_hex(coeff)},")
    print("],")

def print_taylor_coeffs_dyad(poly):
    print("[")
    for i in range(0, 24):
        coeff = poly[i]
        print_dyadic(coeff)
    print("],")

print(f"pub(crate) static J1_COEFFS_TAYLOR: [J1TaylorExtendedSeries; {len(j1_zeros)}] = [")

for i in range(0, len(j1_zeros)):
    k_range = j1_zeros[i]
    range_diff = k_range - prev_zero
    g_c = 1

    x0 = mp.mpf(k_range)
    from mpmath import mp, j1, taylor
    poly = taylor(lambda val: j1(val), x0, 25)
    print_taylor_coeffs(poly, CDR(k_range))
    prev_zero = j1_zeros[i]

print("];")

# print(f"pub(crate) static J1_COEFFS_RATIONAL128: [DyadicFloat128; {len(j1_zeros)}] = [")

# for i in range(0, len(j1_zeros)):
#     k_range = j1_zeros[i]
#     range_diff = k_range - prev_zero
#     g_c = 1

#     x0 = mp.mpf(k_range)
#     from mpmath import mp, j1, taylor
#     poly = taylor(lambda val: j1(val), x0, 25)
#     print_taylor_coeffs_dyad(poly)
#     prev_zero = j1_zeros[i]

# print("];")


Zero 1: x ≈ 3.8317059702075123156144358863081609582911078521244950401886
Zero 2: x ≈ 7.0155866698156187535370499814765222133211799657697498562727
Zero 3: x ≈ 10.173468135062722077185711776775842874118325466668199285671
Zero 4: x ≈ 13.323691936314223032393684126947876836246966256828906387054
Zero 5: x ≈ 16.47063005087763281255246047098955460257890840875283477081
Zero 6: x ≈ 19.615858510468242021125065884137511712322940855970695228805
Zero 7: x ≈ 22.760084380592771898053005152182260032194822293609582761429
Zero 8: x ≈ 25.903672087618382625495855445979875220850924725526971473225
Zero 9: x ≈ 29.046828534916855066647819883531963887188166731590534232106
Zero 10: x ≈ 32.189679910974403626622984104460371875478032041055034115317
Zero 11: x ≈ 35.332307550083865102634479022519015000383928144530784171094
Zero 12: x ≈ 38.47476623477161511205219755771661236717103805681843387566
Zero 13: x ≈ 41.617094212814450885863516805060288649321765027325910311976
Zero 14: x ≈ 44.759318997652821732779352713212144

In [20]:
def print_expansion_at_0():
    print(f"static C: [DyadicFloat128; 13] = ")
    from mpmath import mp, j1, taylor, expm1
    poly = taylor(lambda val: j1(val), 0, 26)
    # print(poly)
    real_i = 0
    print("[")
    for i in range(1, len(poly), 2):
        print_dyadic(poly[i])
        real_i = real_i + 1
    print("],")

    print("];")

mp.prec = 180

print_expansion_at_0()

pub(crate) static C: DyadicFloat128 = [
[
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -128,
    mantissa: 0x80000000_00000000_00000000_00000000_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -131,
    mantissa: 0x80000000_00000000_00000000_00000000_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -136,
    mantissa: 0xaaaaaaaa_aaaaaaaa_aaaaaaaa_aaaaaaab_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -142,
    mantissa: 0xe38e38e3_8e38e38e_38e38e38_e38e38e4_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -148,
    mantissa: 0xb60b60b6_0b60b60b_60b60b60_b60b60b6_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -155,
    mantissa: 0xc22e4506_72894ab6_cd8efb11_d33f5618_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -162,
    mantissa: 0x93f27dbb_c4fae397_780b69f5_333c725b_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -170,
    mantissa: 0xa91

In [3]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = (zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.05
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(j1_zeros)
print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 500;
bessel_j1 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_j1(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d, 1, 1e-25);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RR(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

degree = 13

print(f"pub(crate) static J1F_COEFFS: [[u64;{degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print("[")
    for c in coeffs:
        print(double_to_hex(c) + ",")
    print("],")
print("];")

[(-0.97059, 1.0453, 1.8411837813406593026436295136444), (-1.0453, 0.79987, 3.8317059702075123156144358863082), (-0.79987, 0.89207, 5.3314427735250326368840161834339), (-0.89207, 0.81036, 7.0155866698156187535370499814765), (-0.81036, 0.86858, 8.5363163663462858343589608864121), (-0.86858, 0.81627, 10.173468135062722077185711776776), (-0.81627, 0.85884, 11.706004902592064100192670477240), (-0.85884, 0.81995, 13.323691936314223032393684126948), (-0.81995, 0.85352, 14.863588633909033105004905304986), (-0.85352, 0.82245, 16.470630050877632812552460470990), (-0.82245, 0.85017, 18.015527862681803727025625542010), (-0.85017, 0.82426, 19.615858510468242021125065884137), (-0.82426, 0.84786, 21.164369859188790027767973293705), (-0.84786, 0.82562, 22.760084380592771898053005152182), (-0.82562, 0.84617, 24.311326857210776091733221673754), (-0.84617, 0.82669, 25.903672087618382625495855445980), (-0.82669, 0.84489, 27.457050571059245888223275587902), (-0.84489, 0.82755, 29.04682853491685506664781988

KeyboardInterrupt: 

In [4]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = (zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.05
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(j1_zeros)
# print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 500;
bessel_j1 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_j1(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RealField(500)(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

def print_remez_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        print_dyadic(coeff)
        # print_double_double("", coeff)
    print("],")

degree = 23

print(f"pub(crate) static J1_COEFFS_RATIONAL128: [[(u64, u64); {degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print_remez_coeffs(coeffs)
print("];")

];


In [6]:
def print_expansion_at_0_f():
    print(f"pub(crate) const J1_MACLAURIN_SERIES: [u64; 9] = [")
    from mpmath import mp, j1, taylor, expm1
    mp.prec = 60
    poly = taylor(lambda val: j1(val), 0, 18)
    z = 0
    for i in range(1, 18, 2):
        print(f"{double_to_hex(poly[i])},")
    print("];")

    print(f"poly {poly}")

print_expansion_at_0_f()

pub(crate) const J1_MACLAURIN_SERIES: [u64; 9] = [
0x3fe0000000000000,
0xbfb0000000000000,
0x3f65555555555555,
0xbf0c71c71c71c71c,
0x3ea6c16c16c16c17,
0xbe3845c8a0ce5129,
0x3dc27e4fb7789f5c,
0xbd4522a43f65486a,
0x3cc2c9758daf5cd0,
];
poly [mpf('0.0'), mpf('0.5'), mpf('0.0'), mpf('-0.0625'), mpf('0.0'), mpf('0.0026041666666666666678'), mpf('0.0'), mpf('-0.000054253472222222222228'), mpf('0.0'), mpf('6.7816840277777777802e-7'), mpf('0.0'), mpf('-5.6514033564814814787e-9'), mpf('0.0'), mpf('3.3639305693342151652e-11'), mpf('0.0'), mpf('-1.5017547184527746284e-13'), mpf('0.0'), mpf('5.2144261057388007941e-16'), mpf('0.0')]


In [9]:
# Taylor series for f32
mp.prec = 60
print(f"pub(crate) static J1F_COEFFS: [[u64; 15]; {len(j1_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffsf(poly, x0):
    print("[")
    for i in range(0, 15):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(j1_zeros)):
    k_range = j1_zeros[i]
    range_diff = k_range - prev_zero
    g_c = 1

    x0 = mp.mpf(k_range)
    from mpmath import mp, j1, taylor, expm1
    # The j1 (Bessel J_1) function from mpmath will compute with mp.dps precision
    # The Taylor series computation will also respect mp.dps
    poly = taylor(lambda val: j1(val), x0, 16)
    # print(poly)
    print_taylor_coeffsf(poly, CDR(k_range))
    prev_zero = j1_zeros[i]

print("];")

pub(crate) static J1F_COEFFS: [[u64; 15]; 47] = [
[mpf('0.58186522428159637935'), mpf('0.0'), mpf('-0.20511071214777319618'), mpf('0.0060589483246034702712'), mpf('0.013801769807956073966'), mpf('-0.00037231709715624585667'), mpf('-0.00039495907354755762771'), mpf('9.2029497991652741433e-6'), mpf('6.2672896817308790948e-6'), mpf('-1.2678573687215959376e-7'), mpf('-6.3255405008454650371e-8'), mpf('1.1251416998452125018e-9'), mpf('4.4195229281806885938e-10'), mpf('-6.9988903007532951552e-12'), mpf('-2.2644917176837842057e-12'), mpf('3.2279649218764984938e-14')]
[
0x3fe29ea3d19f035f,
0x0000000000000000,
0xbfca41115c5df243,
0x3f78d1448e6fed48,
0x3f8c441a2f9de22b,
0xbf386671c18b088a,
0xbf39e2504ddc7608,
0x3ee34ccbca0c75d1,
0x3eda4973784d1087,
0xbe81045322aaab45,
0xbe70fae0da6cdcef,
0x3e13546cef5ed00a,
0x3dfe5ee82e667708,
0xbd9ec80cc8b63c39,
0xbd83eb2e99629b44,
],
[mpf('0.0'), mpf('-0.40275939570255297204'), mpf('0.05255614585697723529'), mpf('0.053410444132724809124'), mpf('-0.0051797192456

In [22]:
from mpmath import mp, j1, taylor, linspace, chebyfit

# Set precision (decimal places)
mp.prec = 107

# Generate degree-46 Taylor series of J1(x) at x = 0
poly = taylor(lambda val: j1(val), mp.mpf('1.8411837813407134767373918293742463'), 25)

def polyeval(coeffs, x, center):
    """Evaluate Taylor polynomial centered at `center` using Horner's method."""
    u = x - center
    # result = mpf(0)
    # for c in reversed(coeffs):
    #     result = result * u + c
    # return result
    result = RealField(53)(0)
    for (index, c) in enumerate(reversed(coeffs)):
        if index < 8:
            result = RealField(53)(result) * RealField(53)(u) + RealField(53)(c)
        else:
            result = RealField(107)(result) * RealField(107)(u) + RealField(107)(c)
    return result

xs = linspace(0.991, 2, 1000)  # 1000 evenly spaced points
max_rel_error = 0
max_abs_error = 0
worst_x = None

for x in xs:
    true_val = j1(x)
    approx_val = polyeval(poly, x, DD('1.8411837813407134767373918293742463'))
    if true_val != 0:
        rel_error = abs((approx_val - true_val) / true_val)
        if rel_error > max_rel_error:
            max_rel_error = rel_error
            worst_x = x

print(f"Max relative error over [0, 1]: {max_rel_error}")
print(f"Occurs at x = {worst_x}")
worst_x = None

for x in xs:
    true_val = j1(x)
    approx_val = polyeval(poly, x, DD('1.8411837813407134767373918293742463'))
    abs_error = abs(approx_val - true_val)
    if abs_error > max_abs_error:
        max_abs_error = abs_error
        worst_x = x

print(f"Max abs error over [0, 1]: {max_rel_error}")
print(f"Occurs at x = {worst_x}")

print(double_to_hex(DD(max_rel_error)))
print(polyeval(poly, mp.mpf('0.9750000000213042'), mp.mpf('1.8411837813407134767373918293742463')))
def make_e(eps, p):
    return float((DD(1) + DD(2)**DD(-p)) / (DD(1) - eps - DD(2) ** DD(p + 1)*eps)) * 2**(-p)
print(double_to_hex(make_e(DD('1.192857060732688978535982892962e-26'), 55)))
print(polyeval(poly, DD('0.9218750000227204'), DD('1.8411837813407134767373918293742463')))

Max relative error over [0, 1]: 1.224074717441528028996975552824e-29
Occurs at x = 0.9909999999999999920063942226989
Max abs error over [0, 1]: 1.224074717441528028996975552824e-29
Occurs at x = 0.9909999999999999920063942226989
0x39ef08b2faac819b
0.4318209025803349383680360368051
0x3c800000003b1143
0.4136748745759469052209620580648
